<a href="https://colab.research.google.com/github/ehdgnsdl/dacon/blob/main/penguin_second.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings(action='ignore')

from os.path import join
import pandas as pd
import numpy as np
import multiprocessing

n_cpus = multiprocessing.cpu_count()

ROOT_PATH = ''
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/penguin/train.csv')
x_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/penguin/test.csv')

In [3]:
data.head()

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
0,0,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.0,15.3,220,MALE,8.30515,-25.19017,5550
1,1,Chinstrap penguin (Pygoscelis antarctica),Dream,No,49.5,19.0,200,MALE,9.63074,-24.34684,3800
2,2,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.1,14.4,210,FEMALE,8.51951,-27.01854,4400
3,3,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,44.5,14.7,214,FEMALE,8.20106,-26.16524,4850
4,4,Gentoo penguin (Pygoscelis papua),Biscoe,No,49.6,16.0,225,MALE,8.38324,-26.84272,5700


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   114 non-null    int64  
 1   Species              114 non-null    object 
 2   Island               114 non-null    object 
 3   Clutch Completion    114 non-null    object 
 4   Culmen Length (mm)   114 non-null    float64
 5   Culmen Depth (mm)    114 non-null    float64
 6   Flipper Length (mm)  114 non-null    int64  
 7   Sex                  111 non-null    object 
 8   Delta 15 N (o/oo)    111 non-null    float64
 9   Delta 13 C (o/oo)    111 non-null    float64
 10  Body Mass (g)        114 non-null    int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 9.9+ KB


In [5]:
data['Flipper Length (mm)'] = data['Flipper Length (mm)'].astype('float')
data['Flipper Length (mm)'].dtype

dtype('float64')

In [6]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   228 non-null    int64  
 1   Species              228 non-null    object 
 2   Island               228 non-null    object 
 3   Clutch Completion    228 non-null    object 
 4   Culmen Length (mm)   228 non-null    float64
 5   Culmen Depth (mm)    228 non-null    float64
 6   Flipper Length (mm)  228 non-null    float64
 7   Sex                  222 non-null    object 
 8   Delta 15 N (o/oo)    219 non-null    float64
 9   Delta 13 C (o/oo)    220 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 17.9+ KB


In [7]:
label = data['Body Mass (g)']

data.drop(columns=['id', 'Body Mass (g)'], inplace=True)
x_test.drop(columns=['id'], inplace=True)

In [8]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t =='O']
num_columns = [c for c in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n\n Numeric Columns: \n{}\n'.format(cat_columns, num_columns))

Categorical Columns: 
['Species', 'Island', 'Clutch Completion', 'Sex']

 Numeric Columns: 
['Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Delta 15 N (o/oo)', 'Delta 13 C (o/oo)']



In [9]:
# data 수치형 변수들의 결측치 확인
pd.isna(data[num_columns]).sum().sum()

6

In [10]:
# x_test 수치형 변수들의 결측치 확인
pd.isna(x_test[num_columns]).sum().sum()

17

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(data, label,
                                                      test_size = 0.3,
                                                      random_state=42,
                                                      shuffle=True)
x_train.reset_index(drop=True, inplace=True)
x_valid.reset_index(drop=True, inplace=True)

In [13]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

mice = IterativeImputer(random_state=0)
x_train[num_columns] = mice.fit_transform(x_train[num_columns])
x_valid[num_columns] = mice.transform(x_valid[num_columns])
x_test[num_columns] = mice.transform(x_test[num_columns])

In [14]:
pd.isna(x_train[num_columns]).sum().sum()

0

In [15]:
pd.isna(x_valid[num_columns]).sum().sum()

0

In [16]:
pd.isna(x_test[num_columns]).sum().sum()

0

In [17]:
# 범주형 변수 결측치 처리 및 라벨인코딩
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

ohe = OneHotEncoder(sparse=False)
ohe.fit(data[cat_columns])

OneHotEncoder(sparse=False)

In [18]:
onehot_columns = ohe.categories_[0].tolist()
for col in ohe.categories_[1:]:
  onehot_columns += col.tolist()

In [19]:
x_train_cat = pd.DataFrame(ohe.transform(x_train[cat_columns]), columns=onehot_columns)
x_valid_cat = pd.DataFrame(ohe.transform(x_valid[cat_columns]), columns=onehot_columns)
x_test_cat  = pd.DataFrame(ohe.transform(x_test[cat_columns]),  columns=onehot_columns)

In [20]:
# .values를 해줌으로써 밑에 Xgboost에서 featuer_names is unique라는 에러를 해결
x_train = pd.concat([x_train[num_columns], x_train_cat], axis=1)
x_valid = pd.concat([x_valid[num_columns], x_valid_cat], axis=1)
x_test  = pd.concat([x_test[num_columns],  x_test_cat],  axis=1)

In [21]:
y_train.shape

(79,)

In [36]:
# 라벨 변수 Label Encoding
le = LabelEncoder()
label = le.fit(label)

In [24]:
# 스케일링
from sklearn.preprocessing import StandardScaler
sclaer = StandardScaler()
x_train[num_columns] = sclaer.fit_transform(x_train[num_columns])
x_valid[num_columns] = sclaer.transform(x_valid[num_columns])
x_test[num_columns] = sclaer.transform(x_test[num_columns])

In [25]:
# 스케일링 확인 (std인 표준편차가 1에 가까워야함)
x_train[num_columns].describe()

,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Delta 15 N (o/oo),Delta 13 C (o/oo)
count,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01,7.900000e+01
mean,5.621382e-18,-1.319620e-15,4.314411e-16,8.432074e-16,1.023092e-15
std,1.006390e+00,1.006390e+00,1.006390e+00,1.006390e+00,1.006390e+00
min,-1.948239e+00,-1.980462e+00,-1.906274e+00,-1.991752e+00,-1.559992e+00
25%,-8.526895e-01,-7.281967e-01,-7.871240e-01,-8.470913e-01,-7.869291e-01
50%,7.714643e-02,6.405284e-02,-2.974957e-01,-2.042377e-01,-2.972046e-01
75%,8.780942e-01,7.796331e-01,8.566282e-01,9.924208e-01,8.066664e-01
max,2.028881e+00,2.006342e+00,2.010752e+00,2.280513e+00,1.903624e+00


In [26]:
def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-pred)))
    return score

In [27]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 16 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Culmen Length (mm)                         79 non-null     float64
 1   Culmen Depth (mm)                          79 non-null     float64
 2   Flipper Length (mm)                        79 non-null     float64
 3   Delta 15 N (o/oo)                          79 non-null     float64
 4   Delta 13 C (o/oo)                          79 non-null     float64
 5   Adelie Penguin (Pygoscelis adeliae)        79 non-null     float64
 6   Chinstrap penguin (Pygoscelis antarctica)  79 non-null     float64
 7   Gentoo penguin (Pygoscelis papua)          79 non-null     float64
 8   Biscoe                                     79 non-null     float64
 9   Dream                                      79 non-null     float64
 10  Torgersen                   

In [41]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
score_List=[]

In [42]:
model = KNeighborsRegressor(n_neighbors=5).fit(x_train, y_train)
score_List.append(f'KNeighborsRegressor: train_score: {model.score(x_train, y_train)}, val_score: {model.score(x_valid, y_valid)}')

model = LinearRegression().fit(x_train, y_train)
score_List.append(f'LinearRegression: train_score: {model.score(x_train, y_train)}, val_score: {model.score(x_valid, y_valid)}')

model = Ridge().fit(x_train, y_train)
score_List.append(f'Ridge: train_score: {model.score(x_train, y_train)}, val_score: {model.score(x_valid, y_valid)}')

model = Lasso().fit(x_train, y_train)
score_List.append(f'Lasso: train_score: {model.score(x_train, y_train)}, val_score: {model.score(x_valid, y_valid)}')

model = DecisionTreeRegressor().fit(x_train, y_train)
score_List.append(f'DecisionTreeRegressor: train_score: {model.score(x_train, y_train)}, val_score: {model.score(x_valid, y_valid)}')

model = RandomForestRegressor().fit(x_train, y_train)
score_List.append(f'RandomForestRegressor: train_score: {model.score(x_train, y_train)}, val_score: {model.score(x_valid, y_valid)}')

model = GradientBoostingRegressor().fit(x_train, y_train)
score_List.append(f'GradientBoostingRegressor: train_score: {model.score(x_train, y_train)}, val_score: {model.score(x_valid, y_valid)}')

model = XGBRegressor().fit(x_train, y_train)
score_List.append(f'XGBRegressor: train_score: {model.score(x_train, y_train)}, val_score: {model.score(x_valid, y_valid)}')

model = LGBMRegressor().fit(x_train, y_train)
score_List.append(f'LGBMRegressor: train_score: {model.score(x_train, y_train)}, val_score: {model.score(x_valid, y_valid)}')

[03:21:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [43]:
[print(i) for i in score_List]

KNeighborsRegressor: train_score: 0.8849189955988768, val_score: 0.7497053726169844
LinearRegression: train_score: 0.8721985093781489, val_score: 0.8001381308457658
Ridge: train_score: 0.8697086833082444, val_score: 0.7936489828838199
Lasso: train_score: 0.8720529070156923, val_score: 0.7997623038528142
DecisionTreeRegressor: train_score: 1.0, val_score: 0.3217892786708899
RandomForestRegressor: train_score: 0.9754065404305767, val_score: 0.6825782137094365
GradientBoostingRegressor: train_score: 0.9944569050898459, val_score: 0.6294465221279213
XGBRegressor: train_score: 0.9819615673475177, val_score: 0.7478546508364203
LGBMRegressor: train_score: 0.8932148222626578, val_score: 0.7273957346445322


[None, None, None, None, None, None, None, None, None]

In [44]:
model_LR = LinearRegression().fit(x_train, y_train)
model_RID = Ridge().fit(x_train, y_train)
model_LA = Lasso().fit(x_train, y_train)
model_LGBM = LGBMRegressor().fit(x_train, y_train)
model_XGB = XGBRegressor().fit(x_train, y_train)

[03:21:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [45]:
pred_LR = model_LR.predict(x_test)
pred_RID = model_RID.predict(x_test)
pred_LA = model_LA.predict(x_test)
pred_LGBM = model_LGBM.predict(x_test)
pred_XGB = model_XGB.predict(x_test)
pred = pred_LR*0.2 + pred_RID*0.2 + pred_LA*0.2 + pred_LGBM*0.2 + pred_XGB*0.2
pred

array([4228.48784154, 5609.12730466, 3572.24269651, 3425.11371218,
       3428.88115701, 4143.26463609, 4530.92938354, 5102.23888001,
       4799.47575603, 3912.4812323 , 4131.399232  , 4545.74911334,
       4786.31789525, 4720.82856769, 4939.01879213, 5139.51936389,
       4154.18451475, 3879.2463362 , 3483.40785881, 3548.06455298,
       4048.9657466 , 5504.11157567, 5332.08637156, 4099.82330594,
       3436.29003294, 4560.17410937, 3512.94945018, 3967.14847324,
       4040.06997535, 4592.51306444, 3532.39273962, 4801.69019492,
       3450.71337321, 4962.81094357, 3977.77336476, 3374.06046651,
       4497.39240547, 3619.27548537, 3916.56949828, 4531.74058455,
       4151.99514317, 3553.71169252, 3969.07451611, 5363.06521705,
       3685.15721341, 4520.04395632, 3755.48492132, 4181.4369447 ,
       4801.06683958, 5500.00554603, 3692.12283677, 3981.37072706,
       3441.22526815, 4096.95254126, 4207.42840764, 4013.08138962,
       4548.46434762, 3982.98795527, 4800.95246353, 3642.93222

In [28]:
# # 모델 검증 + OOF 앙상블
# from sklearn.model_selection import KFold
# n_splits = 5
# kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

In [29]:
# # 회귀 모델 사용해보기
# from sklearn.linear_model import LinearRegression
# from sklearn.neural_network import MLPRegressor
# from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor

# val_scores = list()
# #oof_pred = np.zeros((x_test.shape[0], label.shape[0])

# models = [['Linear Regression', LinearRegression(n_jobs=n_cpus)],
#           ['MultiLayerPerceptron', MLPRegressor(random_state=42)],
#           ['RandomForest', RandomForestRegressor(random_state=42,
#                                                  n_jobs=n_cpus)],
#           ['XGBoost', XGBRegressor(random_state=42,
#                                    tree_method='gpu_hist',
#                                    n_jobs=n_cpus)],
#           ['LightGBM', LGBMRegressor(random_state=42,
#                                      tree_method='gpu_hist',
#                                      n_jobs=n_cpus)]]

# best_loss = np.inf
# best_pred = None
# for i, (name, model) in enumerate(models):
#   current_loss = list()
#   current_pred = list()
  
#   model.fit(x_train, y_train)
#   y_pred = model.predict(x_valid)
#   current_loss.append(RMSE(y_valid, y_pred))
#   current_pred.append(model.predict(x_test))

#   if np.mean(current_loss) < best_loss:
#       best_loss  = np.mean(current_loss)
#       best_model = [i, name]
#       best_pred  = current_pred
#       #oof_pred += model.predict(x_test) / n_splits
    
#   print(f'{name} Model Validation RMSE: {np.mean(current_loss)}')

# print(f'Best Model: {best_model[-1]}, RMSE: {best_loss}')

Linear Regression Model Validation RMSE: 345.59930641040074
MultiLayerPerceptron Model Validation RMSE: 4632.256876832222
RandomForest Model Validation RMSE: 441.50552698368665
[03:18:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost Model Validation RMSE: 386.85679306184494
LightGBM Model Validation RMSE: 403.621426772355
Best Model: Linear Regression, RMSE: 345.59930641040074


In [30]:
# # best_pred를 np.array()로 만든 후에 (list를 array로 만들어줌.)
# best_pred = np.array(best_pred)
# best_pred.shape

(1, 228)

In [31]:
# best_pred = best_pred.reshape(-1, 1)
# best_pred.shape

(228, 1)

In [46]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/penguin/sample_submission.csv')
submit.head()

,id,Body Mass (g)
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [47]:
submit['Body Mass (g)'] = pred

In [48]:
submit_file_name = 'penguin_second.csv'
submit.to_csv(submit_file_name, index=False)